# Conditional GMM
Created by: Daniel L. Marino (marinodl@vcu.edu)

Description: gaussian mixture model implementation on tensorflow, the objective is a conditional probabilistic distribution. 

This file serves as a visual test for the gmm extension modules

In [1]:
import math
import numpy as np
import collections
from time import time
import tensorflow as tf
from twodlearn.tf_lib.datasets.generic import Datasets
from twodlearn.tf_lib.GMM import *

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## 1. Create dataset

In [2]:

def gmm_sampling(mu, sigma, w, n_samples=1):
    # generates random vectors, sampled from a gausian mixture model
    #  
    #     - mu: 3d tensor containing the means for the gaussians.
    #           the "depth" dimention (3rd) is used to index the
    #           gaussians.    [ kernel_id, dim]
    #     - sigma: 3d tensor containing the covariance matrix of the
    #              gaussians. [ kernel_id, dim] for diagonal matrices
    #     - w: vector in form of a 3d tensor containing the weights
    #          for each one of the gaussians, they have to sum one. 
    #          [kernel_id]
    n_kernels = mu.shape[0]
    n_dim = mu.shape[1]
    
    # random sample the kernel from which the output samples are going
    # to be drawn
    kernel_id= np.argmax(np.random.multinomial(1, w, size=[n_samples]), axis=1 )
    
    out = np.zeros([n_samples, n_dim])
    for i in range(n_samples):
        out[i,:]= np.random.multivariate_normal(mu[kernel_id[i],:], np.diag(sigma[kernel_id[i],:])) # if diagonal
    
    return out;
    
    

In [3]:
n_samples = 1000000

sigma_r = 1.0*np.exp(-0.005*np.linspace(0, 1000, n_samples))
w_r = [1]  # has to sum to one

aux_x= np.linspace(0, 6, n_samples)
aux_y= np.linspace(0, 6, n_samples)

for i in range(n_samples):
    aux_y[i] = np.random.normal(np.sin( aux_x[i] ), sigma_r[i])

# build the dataset
data = Datasets(aux_x, aux_y)
data.normalize()
print('Data shape: ', data.train.x.shape)

# plot
#plt.plot(data.train.x, data.train.y, 'o')

Data shape:  (1000000,)


In [4]:
print(np.sin(aux_x[2]))

1.20000119997e-05


## 2. Model definition

In [5]:
#config = tf.ConfigProto( device_count = {'GPU': 0} )
#sess = tf.InteractiveSession(config = config)
sess = tf.InteractiveSession()

In [6]:
def average_gradients(tower_grads):
    """Calculate the average gradient for each shared variable across all towers.
    Note that this function provides a synchronization point across all towers.
    Args:
        tower_grads: List of lists of (gradient, variable) tuples. The outer list
        is over individual gradients. The inner list is over the gradient
        calculation for each tower.
    Returns:
        List of pairs of (gradient, variable) where the gradient has been averaged
        across all towers.
    """
    average_grads = []
    # 1. loop through each variable
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        # 2. loop through each tower
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)
            
            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)
            
        # Average over the 'tower' dimension.
        grad = tf.concat(0, grads)
        grad = tf.reduce_mean(grad, 0)
        
        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
        
    return average_grads

In [7]:
n_inputs = 1
n_outputs = 1
n_hidden= [4]
n_kernels = 1

with tf.device('/cpu:0'):
    gmm_model = GmmMlpModel( n_inputs, n_outputs, n_hidden, n_kernels, 
                             afunction= tf.sigmoid, diagonal= True, method='tdl')

# Create optimizer
optimizer = tf.train.AdamOptimizer(0.002)
    
# Create models for each gpu
tower_grads = []
train_models = []
with tf.device('/gpu:0'):
    train_model = gmm_model.setup(n_samples/2)
    train_models.append( train_model )
    
    grads = optimizer.compute_gradients(train_model.loss)
    
    tower_grads.append(grads)
    
with tf.device('/gpu:1'):
    train_model = gmm_model.setup(n_samples/2)
    train_models.append( train_model )
    
    grads = optimizer.compute_gradients(train_model.loss)
    
    tower_grads.append(grads)

# average gradients
grads = average_gradients(tower_grads)

# apply gradients
apply_gradient_op = optimizer.apply_gradients(grads);

In [8]:
print('mu shape:', train_model.mu.get_shape())
print('sigma shape:', train_model.sigma.get_shape())
print('w shape:', train_model.w.get_shape())

print('out shape:', train_model.out.get_shape())

print('loss shape:', train_model.loss.get_shape())


mu shape: (500000, 1, 1)
sigma shape: (500000, 1, 1)
w shape: (500000, 1)
out shape: <unknown>
loss shape: <unknown>


## 3. Training

In [9]:
num_steps = 10000 #1000
n_logging = 100
n_test_logg = 10

tf.initialize_all_variables().run()
print('Initialized')

mean_loss= 0
train_accuracy= 0

t0 = time()
for step in range(num_steps):   
    
    _, l1, l2 = sess.run([apply_gradient_op, train_models[0].loss, train_models[1].loss],
                         feed_dict={train_models[0].inputs : np.expand_dims(data.train.x[:n_samples/2],1),
                                    train_models[0].labels : np.expand_dims(data.train.y[:n_samples/2],1),
                                    train_models[1].inputs : np.expand_dims(data.train.x[n_samples/2:],1),
                                    train_models[1].labels : np.expand_dims(data.train.y[n_samples/2:],1),
                                   })
    mean_loss += 0.5*(l1 + l2)
    
    
    if step%n_logging == 0:                
        print(step, ' | loss:', mean_loss/n_logging)
        mean_loss= 0
        
t1 = time()
print('function takes: ', (t1-t0))

Initialized


/home/marinodl/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/marinodl/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/marinodl/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/marinodl/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0  | loss: 0.0129496192932
100  | loss: 1.21944937229
200  | loss: 1.0807075417
300  | loss: 0.862729559541
400  | loss: 0.73187566936
500  | loss: 0.667426918149
600  | loss: 0.610249723196
700  | loss: 0.545957196355
800  | loss: 0.484142828584
900  | loss: 0.432452298999
1000  | loss: 0.383267418742
1100  | loss: 0.320830774903
1200  | loss: 0.203391505629
1300  | loss: -0.017459448576
1400  | loss: -0.285283521712
1500  | loss: -0.517055316865
1600  | loss: -0.697840419412
1700  | loss: -0.826660934687
1800  | loss: -0.905860103965
1900  | loss: -0.947606394291
2000  | loss: -0.971997644305
2100  | loss: -0.989161190987
2200  | loss: -1.00185326159
2300  | loss: -1.01124534726
2400  | loss: -1.01819988608
2500  | loss: -1.02367614269
2600  | loss: -1.02792557359
2700  | loss: -1.03160936713
2800  | loss: -1.03461680651
2900  | loss: -1.03742195606
3000  | loss: -1.03966118097
3100  | loss: -1.04194389701
3200  | loss: -1.04380267382
3300  | loss: -1.04562702537
3400  | loss: -1.047

## 4. Plot

In [10]:
# model for testing
n_test= 100;
test_model = gmm_model.setup(n_test)


In [11]:
x_plot = np.linspace(-2, 2, n_test)

[mu_out, sigma_out, w_out,] = sess.run([ test_model.mu, test_model.sigma, test_model.w ],
                                         feed_dict= {test_model.inputs : np.expand_dims(x_plot, 1)})
#print('mu:', mu_out, 'sigma:', sigma_out, 'w:', w_out)

print(np.squeeze(mu_out).shape)
print(x_plot.shape)

# plot
#plt.plot(x_plot, np.squeeze(mu_out), 'o')

#plt.plot(x_plot, np.squeeze(mu_out) + np.squeeze(np.sqrt(sigma_out)), 'ro')
#plt.plot(x_plot, np.squeeze(mu_out) - np.squeeze(np.sqrt(sigma_out)), 'ro')


(100,)
(100,)
